# GoREAL API Testing Suite

This notebook provides comprehensive testing for all GoREAL API endpoints.

In [ ]:
import requests
import json
import time
from datetime import datetime
import pandas as pd
import random

# API Configuration
API_BASE_URL = "http://api:5000"  # Docker service name
# For local testing, use: API_BASE_URL = "http://localhost:5000"

# Test data
TEST_PLAYER_ID = "TEST123"
TEST_PLAYER_NAME = "Test_Player"
TEST_CHALLENGE_ID = "C01"

print(f"Testing API at: {API_BASE_URL}")

## 1. Health Check

In [ ]:
def test_health_check():
    """Test the health check endpoint"""
    try:
        response = requests.get(f"{API_BASE_URL}/health", timeout=10)
        print(f"Status Code: {response.status_code}")
        print(f"Response: {json.dumps(response.json(), indent=2)}")
        return response.status_code == 200
    except Exception as e:
        print(f"Error: {e}")
        return False

print("=== HEALTH CHECK TEST ===")
health_ok = test_health_check()
print(f"Health Check Passed: {health_ok}")

## 2. Challenge Logging Tests

In [ ]:
def test_log_challenge(player_id, player_name, challenge_id):
    """Test the log challenge endpoint"""
    payload = {
        "playerId": player_id,
        "playerName": player_name,
        "challengeId": challenge_id
    }
    
    try:
        response = requests.post(
            f"{API_BASE_URL}/log_challenge",
            json=payload,
            headers={'Content-Type': 'application/json'},
            timeout=10
        )
        print(f"Status Code: {response.status_code}")
        print(f"Response: {json.dumps(response.json(), indent=2)}")
        return response.status_code == 200
    except Exception as e:
        print(f"Error: {e}")
        return False

print("=== LOG CHALLENGE TESTS ===")

# Test valid challenge logging
print("\n--- Test 1: Valid Challenge Log ---")
test1_passed = test_log_challenge(TEST_PLAYER_ID, TEST_PLAYER_NAME, TEST_CHALLENGE_ID)

# Test with missing fields
print("\n--- Test 2: Missing Player Name ---")
invalid_payload = {"playerId": TEST_PLAYER_ID, "challengeId": TEST_CHALLENGE_ID}
response = requests.post(f"{API_BASE_URL}/log_challenge", json=invalid_payload, timeout=10)
print(f"Status Code: {response.status_code} (Expected: 400)")
test2_passed = response.status_code == 400

print(f"\nChallenge Logging Tests Passed: {test1_passed and test2_passed}")

## 3. Challenge Submission Tests

In [ ]:
def test_submit_challenge(player_id, challenge_id, submission_text):
    """Test the submit challenge endpoint"""
    payload = {
        "playerId": player_id,
        "challengeId": challenge_id,
        "submissionText": submission_text
    }
    
    try:
        response = requests.post(
            f"{API_BASE_URL}/submit_challenge",
            json=payload,
            headers={'Content-Type': 'application/json'},
            timeout=10
        )
        print(f"Status Code: {response.status_code}")
        print(f"Response: {json.dumps(response.json(), indent=2)}")
        return response.status_code in [200, 404]  # 404 is valid if no prior log exists
    except Exception as e:
        print(f"Error: {e}")
        return False

print("=== SUBMIT CHALLENGE TESTS ===")

# Wait a moment to ensure the previous log challenge is processed
time.sleep(2)

# Test valid submission
print("\n--- Test 1: Valid Submission ---")
submission_text = "I successfully completed the challenge! My room is now perfectly clean and organized."
test1_passed = test_submit_challenge(TEST_PLAYER_ID, TEST_CHALLENGE_ID, submission_text)

# Test submission without prior challenge log
print("\n--- Test 2: Submission Without Prior Log ---")
test2_passed = test_submit_challenge("NONEXISTENT123", "C99", "This should fail")

print(f"\nChallenge Submission Tests Passed: {test1_passed and test2_passed}")

## 4. Status Query Tests

In [ ]:
def test_get_status(player_id, challenge_id):
    """Test the get status endpoint"""
    try:
        response = requests.get(
            f"{API_BASE_URL}/get_status",
            params={'playerId': player_id, 'challengeId': challenge_id},
            timeout=10
        )
        print(f"Status Code: {response.status_code}")
        print(f"Response: {json.dumps(response.json(), indent=2)}")
        return response.status_code in [200, 404]
    except Exception as e:
        print(f"Error: {e}")
        return False

print("=== STATUS QUERY TESTS ===")

# Test status query for existing challenge
print("\n--- Test 1: Query Existing Challenge Status ---")
test1_passed = test_get_status(TEST_PLAYER_ID, TEST_CHALLENGE_ID)

# Test status query for non-existent challenge
print("\n--- Test 2: Query Non-existent Challenge ---")
test2_passed = test_get_status("FAKE123", "C999")

# Test with missing parameters
print("\n--- Test 3: Missing Parameters ---")
response = requests.get(f"{API_BASE_URL}/get_status", params={'playerId': TEST_PLAYER_ID}, timeout=10)
print(f"Status Code: {response.status_code} (Expected: 400)")
test3_passed = response.status_code == 400

print(f"\nStatus Query Tests Passed: {test1_passed and test2_passed and test3_passed}")

## 5. Challenge List Tests

In [ ]:
def test_get_challenges():
    """Test the get challenges endpoint"""
    try:
        response = requests.get(f"{API_BASE_URL}/get_challenges", timeout=10)
        print(f"Status Code: {response.status_code}")
        data = response.json()
        print(f"Number of challenges: {len(data.get('challenges', []))}")
        print(f"Sample response: {json.dumps(data, indent=2)[:500]}...")
        return response.status_code == 200
    except Exception as e:
        print(f"Error: {e}")
        return False

print("=== GET CHALLENGES TEST ===")
challenges_test_passed = test_get_challenges()
print(f"\nGet Challenges Test Passed: {challenges_test_passed}")

## 6. Load Testing

In [ ]:
def load_test_api(num_requests=10):
    """Perform basic load testing on the API"""
    print(f"Starting load test with {num_requests} requests...")
    
    success_count = 0
    total_time = 0
    
    for i in range(num_requests):
        player_id = f"LOAD_TEST_{i}"
        player_name = f"LoadTest_Player_{i}"
        challenge_id = f"C0{(i % 5) + 1}"  # Rotate through C01-C05
        
        start_time = time.time()
        
        payload = {
            "playerId": player_id,
            "playerName": player_name,
            "challengeId": challenge_id
        }
        
        try:
            response = requests.post(
                f"{API_BASE_URL}/log_challenge",
                json=payload,
                headers={'Content-Type': 'application/json'},
                timeout=10
            )
            
            end_time = time.time()
            request_time = end_time - start_time
            total_time += request_time
            
            if response.status_code == 200:
                success_count += 1
            
            print(f"Request {i+1}: {response.status_code} ({request_time:.3f}s)")
            
        except Exception as e:
            print(f"Request {i+1}: Error - {e}")
    
    avg_time = total_time / num_requests
    success_rate = (success_count / num_requests) * 100
    
    print(f"\n=== LOAD TEST RESULTS ===")
    print(f"Total Requests: {num_requests}")
    print(f"Successful Requests: {success_count}")
    print(f"Success Rate: {success_rate:.1f}%")
    print(f"Average Response Time: {avg_time:.3f}s")
    print(f"Total Time: {total_time:.3f}s")

# Run load test
load_test_api(5)  # Start with 5 requests

## 7. Test Results Summary

In [ ]:
# Create a test results summary
test_results = {
    'Test Category': [
        'Health Check',
        'Challenge Logging',
        'Challenge Submission', 
        'Status Query',
        'Challenge List',
        'Load Testing'
    ],
    'Status': [
        '✅ Passed' if health_ok else '❌ Failed',
        '✅ Passed' if test1_passed and test2_passed else '❌ Failed',
        '✅ Passed' if test1_passed and test2_passed else '❌ Failed',
        '✅ Passed' if test1_passed and test2_passed and test3_passed else '❌ Failed',
        '✅ Passed' if challenges_test_passed else '❌ Failed',
        '✅ Completed' # Load test always completes, success depends on results
    ],
    'Description': [
        'API health and endpoint availability',
        'Challenge logging with validation',
        'Proof submission functionality',
        'Status query with error handling',
        'Challenge list retrieval',
        'Basic performance under load'
    ]
}

results_df = pd.DataFrame(test_results)
print("\n=== GOREAL API TEST RESULTS SUMMARY ===")
print(results_df.to_string(index=False))

# Overall test status
passed_tests = results_df['Status'].str.contains('Passed').sum()
total_tests = len(results_df) - 1  # Exclude load test from pass/fail count
print(f"\n📊 Overall Test Results: {passed_tests}/{total_tests} tests passed")

if passed_tests == total_tests:
    print("🎉 All API tests passed! The API is functioning correctly.")
else:
    print("⚠️  Some tests failed. Please review the API implementation.")